In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from optuna.integration import TFKerasPruningCallback
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna

In [7]:
def load_data(data_path):
    data=pd.read_csv(data_path)
    X = data.iloc[: ,:-1].to_numpy()
    Y = data['labels'].to_numpy()

    X = X.reshape(X.shape[0], X.shape[1], 1)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

    return x_train, x_test, y_train, y_test

In [8]:
x_train, x_test, y_train, y_test = load_data("../data/data_proccesed_50.csv")
def create_model(trial):
    
    lstm_units = trial.suggest_int('lstm_units', 32, 128)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    l1_reg = trial.suggest_float('l1_reg', 1e-5, 1e-2, log=True)
    l2_reg = trial.suggest_float('l2_reg', 1e-5, 1e-2, log=True)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)


    normalizer = layers.Normalization()
    normalizer.adapt(x_train)
    
    model = models.Sequential([
        layers.Input(shape=(x_train.shape[1], x_train.shape[2])),
        normalizer,
        layers.LSTM(lstm_units, activation="tanh", 
                    kernel_regularizer=tf.keras.regularizers.L1L2(l1=l1_reg, l2=l2_reg)),
        layers.Dropout(dropout_rate),
        layers.Dense(1, activation="sigmoid"),
    ])
    

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy'],
    )
    
    return model

def objective(trial):
    model = create_model(trial)
    

    history = model.fit(
        x_train, y_train,
        epochs=10,
        batch_size=64,
        validation_data=(x_test, y_test),
        callbacks=[TFKerasPruningCallback(trial, 'val_loss')],
        verbose=0 
    )
    

    val_loss, val_accuracy = model.evaluate(x_test, y_test, verbose=0)
    return val_loss

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, timeout=600)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

best_trial = study.best_trial
model = create_model(best_trial)
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(x_test, y_test)
)

model.save('best_model.h5')


[I 2024-05-26 20:40:00,857] A new study created in memory with name: no-name-2277e5e8-1b34-42e5-9000-f1370aa3dc5d
C:\Users\Kuba\AppData\Local\Temp\ipykernel_11468\1097834471.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
[W 2024-05-26 20:41:16,294] Trial 0 failed with parameters: {'lstm_units': 89, 'dropout_rate': 0.4098300668379151, 'l1_reg': 0.0001627104699568199, 'l2_reg': 0.00045468286690533455, 'learning_rate': 0.00387354650342964} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Kuba\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Kuba\AppData\Local

KeyboardInterrupt: 